# Development and Diversity of Bangalore

City of Bangalore has developed very Rapidly over the last couple of decades  
which made it very diverse - but is the diversity evenly distributed across the city?  
or is the development has triggered certain changes which even though looking diverse - is not that diverse  

To set the business in Bangalore it's very much required to understand the diversity  
the business problem that we are going to look into with this study would be to understand the food preferences  
the assumptions being made is to look at the food preferences from the presence of that category of food outlets in the area  

### Business Problem
if you are setting up a new food outlet, the business problem for the new establishment to understand the diversity and what kind of food outlet should be opened   
in which part of the city, and understand the current food preferences of the area

## Data Source
### Area Information
For Finding out areas of bangalore government of india's open dataset - https://data.gov.in/catalog/locality-based-pincode
can be used to fetch the areas of Bangalore

### Location Information
Geopy doesn't provide for the location of the areas within bangalore it provides for the location of bangalore itself but for all other areas it give the latitude longitude of bangalore itself so Geopy is not good source of information there but can be used to fetch the coordinates of Bangalore

### Alternate source of Location information
one of the dataset of bangalore at https://data.gov.in/catalog/locality-based-pincode provides of latitude and longitude information - it doesn't provide location information for rural areas of bangalore but for urban areas of bangalore it provides for good amount of information and especially for the two regions which have grown up recently it provide of the location information - so we will be using the data set for that

### About the dataset
you can use the temporary key to fetch the subset of information
but if you want to have a full look at the full data you would need to login to the website
a key is provided for each user which can be used to fetch the data from the API the data can be fetched in xml/json/csv format for our analysid we would be fetching the csv format data

### Processing and Analysis
form Foursquare we would be fetching the food outlets around in the postal code area. as we are not comparing the neighbourhood
but looking at the food preferences we would be increaing our radius of search to 3000

In [2]:
import pandas as pd

In [3]:
url = "https://api.data.gov.in/resource/04cbe4b1-2f2b-4c39-a1d5-1c2e28bc0e32?api-key=<key>&format=csv&offset=0&limit=1000000"

In [4]:
df = pd.read_csv(url)

In [5]:
df.to_csv("postal.csv")

In [6]:
#import libraries
import urllib.request
import pandas as pd
# import k-means from clustering stage
from sklearn.cluster import KMeans
import requests
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    mkl_fft-1.0.6              |           py36_0         586 KB  conda-forge
    mkl_random-1.0.2           |           py36_0         1.3 MB  conda-forge
    mkl-2018.0.3               |                1       198.7 MB
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    numpy-1.15.0               |   py36h1b885b7_0          35 KB
    branca-0.3.1               |             py_0          25 KB  conda-forge
    numpy-base-1.15.0          |   py3

In [7]:
newDF = df.loc[df['Districtname'] == 'Bangalore']

In [8]:
newDF = newDF.loc[newDF['latitude'] >0]

In [9]:
newDF.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
52180,Arabic College S.O,560045,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25432636,NaN,Bangalore G.P.O.,77.6206,13.0291
52187,Bellandur S.O,560103,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore South,Bangalore,KARNATAKA,080-28441625,NaN,H.A.L II Stage H.O,77.6760,12.9298
52201,Domlur S.O,560071,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25356966,NaN,H.A.L II Stage H.O,77.6359,12.9611
52204,Dr. Shivarama Karanth Nagar S.O,560077,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,NaN,NaN,Bangalore G.P.O.,77.6293,13.0681
52206,Fraser Town S.O,560005,S.O,Delivery,Bangalore East,Bangalore HQ,Karnataka,Bangalore North,Bangalore,KARNATAKA,080-25485125,NaN,Bangalore G.P.O.,77.6164,13.0005


In [10]:
geolocator = Nominatim(user_agent="New One")
location = geolocator.geocode("Bangalore")

In [11]:
location

Location(Bengaluru, Bangalore Urban, Karnataka, India, (12.9791198, 77.5912997, 0.0))

In [12]:
map_my = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

map_my

In [13]:
for lat, lng, postal, neighborhood in zip(newDF['latitude'], newDF['longitude'], newDF['pincode'], newDF['officename']):
    label = '{}, {}'.format(postal,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_my)  
    
map_my

Getting Latitude / Longitude of places and storing the data in a file



In [174]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&query=foods&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(name, len(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [175]:
LIMIT = 200
ny_venues = getNearbyVenues(names=newDF['officename'],
                                   latitudes=newDF['latitude'],
                                   longitudes=newDF['longitude']
                                  )

Arabic College S.O
Arabic College S.O 55
Bellandur S.O
Bellandur S.O 65
Domlur S.O
Domlur S.O 100
Dr. Shivarama Karanth Nagar S.O
Dr. Shivarama Karanth Nagar S.O 3
Fraser Town S.O
Fraser Town S.O 43
G.K.V.K. S.O
G.K.V.K. S.O 17
H.A.L II Stage H.O
H.A.L II Stage H.O 100
HighCourt S.O
HighCourt S.O 100
Jeevanbhimanagar S.O
Jeevanbhimanagar S.O 100
Kothanur S.O
Kothanur S.O 1
Mahatma Gandhi Road S.O
Mahatma Gandhi Road S.O 100
Marathahalli Colony S.O
Marathahalli Colony S.O 65
NAL S.O
NAL S.O 100
New Thippasandra S.O
New Thippasandra S.O 100
Sadashivanagar S.O
Sadashivanagar S.O 100
Sahakaranagar P.O S.O
Sahakaranagar P.O S.O 23
Vimanapura S.O
Vimanapura S.O 75
Yelahanka S.O
Yelahanka S.O 4
Yelahanka Satellite Town S.O
Yelahanka Satellite Town S.O 6
Bangalore G.P.O.
Bangalore G.P.O. 100
Ashoknagar S.O (Bangalore)
Ashoknagar S.O (Bangalore) 100
B Sk II Stage S.O
B Sk II Stage S.O 100
Bannerghatta Road S.O
Bannerghatta Road S.O 100
Basavanagudi H.O
Basavanagudi H.O 91
Bommanahalli S.O (Bang

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['PostalCode'] = ny_venues['PostalCode'] 
ny_onehot['Latitude'] = ny_venues['PostalCode Latitude'] 
ny_onehot['Longitude'] = ny_venues['PostalCode Longitude'] 


# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Longitude,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,...,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,PostalCode,Latitude
0,77.6206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Arabic College S.O,13.0291
1,77.6206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Arabic College S.O,13.0291
2,77.6206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Arabic College S.O,13.0291
3,77.6206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Arabic College S.O,13.0291
4,77.6206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Arabic College S.O,13.0291


In [18]:
ny_grouped =ny_onehot.groupby(['PostalCode', 'Latitude','Longitude']).sum().reset_index()
ny_grouped.head()


,PostalCode,Latitude,Longitude,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Museum,Asian Restaurant,BBQ Joint,...,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Arabic College S.O,13.0291,77.6206,1,0,2,0,0,0,3,...,0,0,0,0,0,0,0,0,2,0
1,Ashoknagar S.O (Bangalore),12.9377,77.5645,0,0,1,0,0,4,0,...,0,1,0,0,0,0,0,0,2,0
2,B Sk II Stage S.O,12.9265,77.5695,0,0,1,0,0,6,0,...,0,1,0,0,0,0,0,0,3,0
3,Bangalore G.P.O.,12.9814,77.5955,1,2,1,1,0,3,0,...,1,0,0,1,1,0,0,0,2,0
4,Bannerghatta Road S.O,12.8976,77.5997,0,1,1,0,0,4,1,...,0,0,0,0,0,0,0,0,6,0


In [19]:
num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode','Latitude','Longitude']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['PostalCode'] = ny_grouped['PostalCode']
ny_venues_sorted['Latitude'] = ny_grouped['Latitude']
ny_venues_sorted['Longitude'] = ny_grouped['Longitude']

for ind in np.arange(ny_grouped.shape[0]):
    ny_venues_sorted.iloc[ind, 3:] = return_most_common_venues(ny_grouped.iloc[ind, 2:], num_top_venues)

ny_venues_sorted

,PostalCode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arabic College S.O,13.0291,77.6206,Indian Restaurant,Chinese Restaurant,Café,Korean Restaurant,BBQ Joint,Steakhouse,Andhra Restaurant,Diner,Fast Food Restaurant,Vegetarian / Vegan Restaurant
1,Ashoknagar S.O (Bangalore),12.9377,77.5645,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Fast Food Restaurant,Asian Restaurant,Café,Restaurant,Salad Place,Sandwich Place,Lounge
2,B Sk II Stage S.O,12.9265,77.5695,Indian Restaurant,Chinese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Breakfast Spot
3,Bangalore G.P.O.,12.9814,77.5955,Indian Restaurant,Chinese Restaurant,Hotel,Restaurant,Pub,Café,Asian Restaurant,Italian Restaurant,Middle Eastern Restaurant,Steakhouse
4,Bannerghatta Road S.O,12.8976,77.5997,Indian Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Asian Restaurant,Rajasthani Restaurant,Shopping Mall,Restaurant,Snack Place,Breakfast Spot
5,Basavanagudi H.O,12.9446,77.5741,Indian Restaurant,Chinese Restaurant,Asian Restaurant,Café,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Lounge,Restaurant,Hotel,Sandwich Place
6,Bellandur S.O,12.9298,77.6760,Indian Restaurant,Hotel,Lounge,Food Court,Breakfast Spot,Asian Restaurant,Shopping Mall,Eastern European Restaurant,Park,Café
7,Bommanahalli S.O (Bangalore),12.9068,77.6311,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Burger Joint,Restaurant,Café,Asian Restaurant,Italian Restaurant
8,Bommasandra Industrial Estate S.O,12.8203,77.6878,Indian Restaurant,Hotel,Vietnamese Restaurant,Fast Food Restaurant,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
9,Carmelram S.O,12.9078,77.6937,Indian Restaurant,Lounge,Hotel,Eastern European Restaurant,Shopping Mall,Burrito Place,Punjabi Restaurant,Café,Multiplex,Mediterranean Restaurant


In [104]:
import matplotlib.cm as cm
import matplotlib.colors as colors
def clusters(kclusters, grouped_data):
    global clusteredDF
    ny_grouped_clustering = grouped_data.drop(['PostalCode', 'Latitude', 'Longitude'],1)
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)
    kmeans.labels_[0:]
    tempDF = grouped_data
    tempDF['Cluster Labels'] = kmeans.labels_
    ny_venues_sorted['Cluster Labels'] = kmeans.labels_
    tempDF['Latitude'] = grouped_data['Latitude']
    tempDF['Longitude'] = grouped_data['Longitude']
    # create map

    map_my = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)
        
    x = np.arange(kclusters)
    ys = [i+x+(i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]
    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(tempDF['Latitude'], tempDF['Longitude'], tempDF['PostalCode'], tempDF['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster],
            fill_opacity=0.7).add_to(map_my)

    display(map_my)
    return tempDF


In [96]:
clusteredDF = clusters(2,ny_grouped)

In [97]:
clusteredDF = clusters(3,ny_grouped)

In [113]:
clusteredDF = clusters(4,ny_grouped)

In [114]:
clusteredDF = clusters(5,ny_grouped)

In [115]:
ny_venues_sorted.loc[ny_venues_sorted['Cluster Labels'] == 0]

,PostalCode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
8,Bommasandra Industrial Estate S.O,12.8203,77.6878,Indian Restaurant,Hotel,Vietnamese Restaurant,Fast Food Restaurant,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,0
10,Chandapura S.O,12.8016,77.7041,Indian Restaurant,Vietnamese Restaurant,Food,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,0
13,Dr. Shivarama Karanth Nagar S.O,13.0681,77.6293,Indian Restaurant,Buffet,Shopping Mall,Vietnamese Restaurant,English Restaurant,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,0
16,G.K.V.K. S.O,13.0781,77.5784,Indian Restaurant,Chinese Restaurant,Resort,Café,Italian Restaurant,Supermarket,Coffee Shop,Eastern European Restaurant,Convenience Store,Deli / Bodega,0
18,Gottigere S.O,12.8564,77.5883,Indian Restaurant,Fast Food Restaurant,Italian Restaurant,Dive Bar,Mexican Restaurant,Resort,Rajasthani Restaurant,Beer Garden,Shopping Mall,Multiplex,0
25,Jalahalli East S.O,13.0728,77.5463,Indian Restaurant,Vietnamese Restaurant,Food,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,0
26,Jalahalli H.O,13.0506,77.5539,Indian Restaurant,Fast Food Restaurant,Hotel,Chinese Restaurant,Gourmet Shop,Kerala Restaurant,Burger Joint,Restaurant,Seafood Restaurant,Smoke Shop,0
31,Kothanur S.O,13.0614,77.6478,Shopping Mall,Vietnamese Restaurant,Fast Food Restaurant,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,0
50,Vidyaranyapura S.O,13.0760,77.5575,Indian Restaurant,Café,Vietnamese Restaurant,Food,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,0
53,Yelahanka S.O,13.0958,77.5975,Vegetarian / Vegan Restaurant,Indian Restaurant,Resort,Food Truck,Vietnamese Restaurant,English Restaurant,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,0


In [116]:
ny_venues_sorted.loc[ny_venues_sorted['Cluster Labels'] == 1]

,PostalCode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
7,Bommanahalli S.O (Bangalore),12.9068,77.6311,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Burger Joint,Restaurant,Café,Asian Restaurant,Italian Restaurant,1
17,Gayathrinagar S.O,13.0010,77.5581,Indian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Asian Restaurant,Lounge,Hotel,Food Court,Fast Food Restaurant,Italian Restaurant,Pizza Place,1
34,Malleswaram S.O,13.0002,77.5709,Indian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Chinese Restaurant,Italian Restaurant,Shopping Mall,Rajasthani Restaurant,Fast Food Restaurant,Pub,Restaurant,1
35,Malleswaram West S.O,13.0089,77.5629,Indian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Lounge,Fast Food Restaurant,Italian Restaurant,Hotel,Restaurant,Food Court,Asian Restaurant,1
36,Marathahalli Colony S.O,12.9559,77.6880,Indian Restaurant,Chinese Restaurant,Hotel,Asian Restaurant,BBQ Joint,Vegetarian / Vegan Restaurant,Restaurant,Breakfast Spot,Food Court,Shopping Mall,1
43,Palace Guttahalli S.O,12.9993,77.5819,Indian Restaurant,Vegetarian / Vegan Restaurant,Hotel,BBQ Joint,Shopping Mall,Italian Restaurant,Bakery,Chinese Restaurant,Asian Restaurant,Gourmet Shop,1
44,Rajajinagar H.O,12.9932,77.5545,Indian Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Chinese Restaurant,Food Court,Italian Restaurant,Asian Restaurant,Lounge,Restaurant,Hotel,1
45,Rajajinagar IVth Block S.O,12.9831,77.5612,Indian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Hotel,Fast Food Restaurant,Asian Restaurant,Pizza Place,Restaurant,Miscellaneous Shop,Pub,1
46,Sadashivanagar S.O,13.0094,77.5772,Indian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Asian Restaurant,Pub,Shopping Mall,Restaurant,1
49,Swimming Pool Extn S.O,12.9968,77.5740,Indian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Chinese Restaurant,Italian Restaurant,Shopping Mall,Restaurant,Pub,Fast Food Restaurant,Rajasthani Restaurant,1


In [117]:
ny_venues_sorted.loc[ny_venues_sorted['Cluster Labels'] == 2]

,PostalCode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Arabic College S.O,13.0291,77.6206,Indian Restaurant,Chinese Restaurant,Café,Korean Restaurant,BBQ Joint,Steakhouse,Andhra Restaurant,Diner,Fast Food Restaurant,Vegetarian / Vegan Restaurant,2
6,Bellandur S.O,12.9298,77.6760,Indian Restaurant,Hotel,Lounge,Food Court,Breakfast Spot,Asian Restaurant,Shopping Mall,Eastern European Restaurant,Park,Café,2
9,Carmelram S.O,12.9078,77.6937,Indian Restaurant,Lounge,Hotel,Eastern European Restaurant,Shopping Mall,Burrito Place,Punjabi Restaurant,Café,Multiplex,Mediterranean Restaurant,2
14,Electronics City S.O,12.8400,77.6626,Indian Restaurant,Hotel,Food Court,Asian Restaurant,Food,Burger Joint,Sandwich Place,Breakfast Spot,Café,Middle Eastern Restaurant,2
15,Fraser Town S.O,13.0005,77.6164,Indian Restaurant,Middle Eastern Restaurant,Café,Hotel,Bakery,Steakhouse,Seafood Restaurant,Shopping Mall,Italian Restaurant,Japanese Restaurant,2
32,Mahalakshmipuram Layout S.O,13.0108,77.5466,Indian Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Chinese Restaurant,Lounge,Italian Restaurant,Food Court,Asian Restaurant,Hotel,French Restaurant,2
37,Mathikere S.O,13.0403,77.5576,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Restaurant,Italian Restaurant,Burger Joint,Hotel,Smoke Shop,Gourmet Shop,Asian Restaurant,2
39,Msrit S.O,13.0298,77.5642,Indian Restaurant,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Lounge,Vegetarian / Vegan Restaurant,Hotel,Food Court,Burger Joint,Asian Restaurant,2
42,Padmanabhnagar S.O,12.9176,77.5576,Indian Restaurant,Breakfast Spot,Chinese Restaurant,Café,Fast Food Restaurant,Asian Restaurant,Restaurant,Salad Place,Ice Cream Shop,Italian Restaurant,2
47,Sahakaranagar P.O S.O,13.0621,77.5880,Indian Restaurant,Chinese Restaurant,Café,Resort,Italian Restaurant,Hotel,Shopping Mall,Bakery,Coffee Shop,Supermarket,2


In [118]:
ny_venues_sorted.loc[ny_venues_sorted['Cluster Labels'] == 3]

,PostalCode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1,Ashoknagar S.O (Bangalore),12.9377,77.5645,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Fast Food Restaurant,Asian Restaurant,Café,Restaurant,Salad Place,Sandwich Place,Lounge,3
2,B Sk II Stage S.O,12.9265,77.5695,Indian Restaurant,Chinese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Restaurant,Fast Food Restaurant,Café,South Indian Restaurant,Breakfast Spot,3
4,Bannerghatta Road S.O,12.8976,77.5997,Indian Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Asian Restaurant,Rajasthani Restaurant,Shopping Mall,Restaurant,Snack Place,Breakfast Spot,3
5,Basavanagudi H.O,12.9446,77.5741,Indian Restaurant,Chinese Restaurant,Asian Restaurant,Café,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Lounge,Restaurant,Hotel,Sandwich Place,3
11,Dharmaram College S.O,12.9352,77.5684,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Asian Restaurant,Fast Food Restaurant,Sandwich Place,South Indian Restaurant,Café,Hotel,Restaurant,3
20,HSR Layout S.O,12.9152,77.6382,Indian Restaurant,Chinese Restaurant,Café,Fast Food Restaurant,Italian Restaurant,BBQ Joint,Snack Place,Restaurant,Bistro,Burger Joint,3
23,J P Nagar S.O,12.9119,77.5868,Indian Restaurant,Chinese Restaurant,Asian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,South Indian Restaurant,Fast Food Restaurant,Snack Place,Rajasthani Restaurant,Sandwich Place,3
24,JP Nagar III Phase S.O,12.9139,77.5976,Indian Restaurant,Chinese Restaurant,Restaurant,Italian Restaurant,Asian Restaurant,Fast Food Restaurant,Pub,South Indian Restaurant,Snack Place,Vegetarian / Vegan Restaurant,3
27,Jayanagar H.O,12.9237,77.5860,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Snack Place,Sandwich Place,Shopping Mall,3
28,Jayangar III Block S.O,12.9304,77.5835,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Asian Restaurant,Restaurant,Snack Place,Sandwich Place,Vegetarian / Vegan Restaurant,South Indian Restaurant,Breakfast Spot,3


In [119]:
ny_venues_sorted.loc[ny_venues_sorted['Cluster Labels'] == 4]

,PostalCode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,Bangalore G.P.O.,12.9814,77.5955,Indian Restaurant,Chinese Restaurant,Hotel,Restaurant,Pub,Café,Asian Restaurant,Italian Restaurant,Middle Eastern Restaurant,Steakhouse,4
12,Domlur S.O,12.9611,77.6359,Indian Restaurant,Chinese Restaurant,Pub,Brewery,Vegetarian / Vegan Restaurant,Andhra Restaurant,Mexican Restaurant,Hotel,Café,Asian Restaurant,4
19,H.A.L II Stage H.O,12.9686,77.6427,Indian Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Pub,Lounge,Asian Restaurant,Café,Hotel,Brewery,Mexican Restaurant,4
21,HighCourt S.O,12.9773,77.5915,Indian Restaurant,Chinese Restaurant,Hotel,Restaurant,Pub,Italian Restaurant,Bar,Shopping Mall,Vegetarian / Vegan Restaurant,Brewery,4
22,Hulimavu S.O,12.8850,77.6022,Indian Restaurant,South Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Italian Restaurant,Breakfast Spot,Rajasthani Restaurant,Restaurant,Mediterranean Restaurant,4
29,Jeevanbhimanagar S.O,12.9695,77.6535,Indian Restaurant,Chinese Restaurant,Pub,Vegetarian / Vegan Restaurant,Restaurant,Mexican Restaurant,Lounge,Café,Brewery,Asian Restaurant,4
30,Koramangala VI Bk S.O,12.9384,77.6186,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Café,Pub,Lounge,Asian Restaurant,Shopping Mall,Vegetarian / Vegan Restaurant,4
33,Mahatma Gandhi Road S.O,12.9739,77.6100,Indian Restaurant,Chinese Restaurant,Hotel,Restaurant,Pub,Bar,Asian Restaurant,Seafood Restaurant,Middle Eastern Restaurant,Brewery,4
40,NAL S.O,12.9596,77.6502,Indian Restaurant,Chinese Restaurant,Pub,Mexican Restaurant,Lounge,Vegetarian / Vegan Restaurant,Brewery,Restaurant,Café,Andhra Restaurant,4
41,New Thippasandra S.O,12.9716,77.6572,Indian Restaurant,Chinese Restaurant,Café,Vegetarian / Vegan Restaurant,Pub,Lounge,Restaurant,Mexican Restaurant,Asian Restaurant,Brewery,4
